In [3]:
import pandas as pd
import h5py

In [4]:
FILE_PATH = "C:\\Users\\zodes\\Documents\\Academics\\UChicago MPCS\\Big Data App Arch\\A\\A\\A\\A\\TRAAAAK128F9318786.h5"

# File Structure

In [5]:
def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"Group: {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name} | Shape: {obj.shape} | Data type: {obj.dtype}")

with h5py.File(FILE_PATH, 'r') as h5_file:
    h5_file.visititems(print_structure)

Group: analysis
Dataset: analysis/bars_confidence | Shape: (87,) | Data type: float64
Dataset: analysis/bars_start | Shape: (87,) | Data type: float64
Dataset: analysis/beats_confidence | Shape: (350,) | Data type: float64
Dataset: analysis/beats_start | Shape: (350,) | Data type: float64
Dataset: analysis/sections_confidence | Shape: (6,) | Data type: float64
Dataset: analysis/sections_start | Shape: (6,) | Data type: float64
Dataset: analysis/segments_confidence | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_max | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_max_time | Shape: (585,) | Data type: float64
Dataset: analysis/segments_loudness_start | Shape: (585,) | Data type: float64
Dataset: analysis/segments_pitches | Shape: (585, 12) | Data type: float64
Dataset: analysis/segments_start | Shape: (585,) | Data type: float64
Dataset: analysis/segments_timbre | Shape: (585, 12) | Data type: float64
Dataset: analysis/songs | Shape: (1,)

# Extracting to Pandas Dataframe

In [28]:
h5 = h5py.File(FILE_PATH, 'r')
store = pd.HDFStore(FILE_PATH, mode = 'r')

In [39]:
songs_metadata = store.get("metadata/songs")
songs_metadata

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,166043,,0,0,Adelitas Way,497103,0.733372,SOBLFFE12AF72AA5BA,Scream,5504670


In [ ]:
songs_analysis = store.get("analysis/songs")
songs_analysis

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,7b6557889cc46b202300a4baf902e229,0.0,213.9424,0.0,0.0,0,0,0,0,...,1,0.349,-4.769,1,0.403,210.048,99.944,4,0.846,TRAAAAK128F9318786


In [31]:
songs_year = store.get("musicbrainz/songs")
songs_year

,idx_artist_mbtags,year
0,0,2009


In [ ]:
bars_confidence = pd.Series([[item for item in list(h5.get("analysis/bars_confidence"))]], name = "bars_confidence")
bars_confidence

0    [0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...
Name: bars_confidence, dtype: object

In [37]:
segments_pitches = pd.Series([[item for item in list(h5.get("analysis/segments_pitches"))]], name = "segments_pitches")
segments_pitches

0    [[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18...
Name: segments_pitches, dtype: object

In [ ]:
song = pd.concat([songs_metadata, songs_analysis, songs_year, bars_confidence, segments_pitches], axis=1, join='outer')
song

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id,idx_artist_mbtags,year,bars_confidence,segments_pitches
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,...,0.403,210.048,99.944,4,0.846,TRAAAAK128F9318786,0,2009,"[0.1, 0.063, 0.319, 0.002, 0.01, 0.015, 0.182,...","[[0.294, 0.158, 0.242, 0.182, 0.2, 0.407, 0.18..."


In [40]:
song = pd.concat([
    store.get("metadata/songs"),
    store.get("analysis/songs"),
    store.get("musicbrainz/songs"),
    pd.Series([[item for item in list(h5.get("analysis/bars_confidence"))]], name = "bars_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/bars_start"))]], name = "bars_start"),
    pd.Series([[item for item in list(h5.get("analysis/beats_confidence"))]], name = "beats_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/beats_start"))]], name = "beats_start"),
    pd.Series([[item for item in list(h5.get("analysis/sections_confidence"))]], name = "sections_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/sections_start"))]], name = "sections_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_confidence"))]], name = "segments_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_max"))]], name = "segments_loudness_max"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_max_time"))]], name = "segments_loudness_max_time"),
    pd.Series([[item for item in list(h5.get("analysis/segments_loudness_start"))]], name = "segments_loudness_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_pitches"))]], name = "segments_pitches"),
    pd.Series([[item for item in list(h5.get("analysis/segments_start"))]], name = "segments_start"),
    pd.Series([[item for item in list(h5.get("analysis/segments_timbre"))]], name = "segments_timbre"),
    pd.Series([[item for item in list(h5.get("analysis/tatums_confidence"))]], name = "tatums_confidence"),
    pd.Series([[item for item in list(h5.get("analysis/tatums_start"))]], name = "tatums_start"),
    pd.Series([[item for item in list(h5.get("metadata/artist_terms"))]], name = "artist_terms"),
    pd.Series([[item for item in list(h5.get("metadata/artist_terms_freq"))]], name = "artist_terms_freq"),
    pd.Series([[item for item in list(h5.get("metadata/artist_terms_weight"))]], name = "artist_terms_weight"),
    pd.Series([[item for item in list(h5.get("metadata/similar_artists"))]], name = "similar_artists"),
    pd.Series([[item for item in list(h5.get("musicbrainz/artist_mbtags"))]], name = "artist_mbtags"),
    pd.Series([[item for item in list(h5.get("musicbrainz/artist_mbtags_count"))]], name = "artist_mbtags_count")
], axis=1, join='outer')
song

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,...,segments_start,segments_timbre,tatums_confidence,tatums_start,artist_terms,artist_terms_freq,artist_terms_weight,similar_artists,artist_mbtags,artist_mbtags_count
0,,324573,0.639903,0.461318,ARJNIUY12298900C91,NaN,,NaN,6ae6a016-91d7-46cc-be7d-5e8e5d320c54,Adelitas Way,...,"[0.0, 0.54286, 0.70553, 1.48367, 1.68063, 1.95...","[[17.977, 71.335, 193.384, -323.599, 25.153, 7...","[0.429, 0.39, 0.355, 0.328, 0.303, 0.277, 0.24...","[0.10184, 0.25966, 0.41592, 0.57454, 0.73158, ...","[b'alternative metal', b'heavy metal', b'rock'...","[0.9958746574225641, 0.9958746574225641, 1.0, ...","[1.0, 0.8993537393186563, 0.7561876828863154, ...","[b'AROQMHX11C8A42BD89', b'ARZEKVZ1187FB51D85',...",[],[]
